In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive/bert-data

ln: failed to create symbolic link '/mydrive/My Drive': File exists
employee_review_mturk_dataset_test_v6_kaggle.csv  employee_review_mturk_dataset_v10_kaggle.csv


In [5]:
data_folder = '/mydrive/bert-data'

In [1]:
import pandas as pd
import nltk
from nltk import tokenize

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
pass
import re
from nltk.corpus import stopwords
import re
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
df = pd.read_csv(data_folder + f'/employee_review_mturk_dataset_v10_kaggle.csv')
df.set_index('id', inplace=True)
df.dropna(inplace=True)

In [7]:
df.head(5)

,person_name,nine_box_category,feedback,adjusted,reviewed
id,,,,,
1,John Doe,"Category 1: 'Risk' (Low performance, Low poten...",John has not progressed in his position. He is...,False,True
2,John Doe,"Category 1: 'Risk' (Low performance, Low poten...",John has consistently disappointed me this qua...,False,True
3,John Doe,"Category 1: 'Risk' (Low performance, Low poten...",John turned in subpar work product all quarter...,False,True
6,John Doe,"Category 1: 'Risk' (Low performance, Low poten...",John Doe demonstrates a low level of knowledge...,False,True
7,George Gill,"Category 1: 'Risk' (Low performance, Low poten...",George gill's performance is really poor. He d...,False,True


In [8]:
df.nine_box_category.value_counts().sort_index()

,count
nine_box_category,
"Category 1: 'Risk' (Low performance, Low potential)",126
"Category 2: 'Average performer' (Moderate performance, Low potential)",106
"Category 3: 'Solid Performer' (High performance, Low potential)",89
"Category 4: 'Inconsistent Player' (Low performance, Moderate potential)",113
"Category 5: 'Core Player' (Moderate performance, Moderate potential)",112
"Category 6: 'High Performer' (High performance, Moderate potential)",91
"Category 7: 'Potential Gem' (Low performance, High potential)",41
"Category 8: 'High Potential' (Moderate performance, High potential)",88
"Category 9: 'Star' (High performance, High potential)",112


In [9]:
stop_words = set(stopwords.words('english'))

In [10]:
def clean_text(text):
    text = re.sub(r"[^a-zA-Z0-9\s]", '', text)
    text = re.sub(r"\s+", ' ', text)
    text = text.lower()

    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

df['cleaned_feedback'] = df['feedback'].apply(clean_text)

In [11]:
df['label'] = df['nine_box_category'].str.extract(r'Category (\d+):')[0].astype(int) - 1

In [12]:
df

,person_name,nine_box_category,feedback,adjusted,reviewed,cleaned_feedback,label
id,,,,,,,
1,John Doe,"Category 1: 'Risk' (Low performance, Low poten...",John has not progressed in his position. He is...,False,True,john progressed position continuously late lea...,0
2,John Doe,"Category 1: 'Risk' (Low performance, Low poten...",John has consistently disappointed me this qua...,False,True,john consistently disappointed quarter shoddy ...,0
3,John Doe,"Category 1: 'Risk' (Low performance, Low poten...",John turned in subpar work product all quarter...,False,True,john turned subpar work product quarter asked ...,0
6,John Doe,"Category 1: 'Risk' (Low performance, Low poten...",John Doe demonstrates a low level of knowledge...,False,True,john doe demonstrates low level knowledge requ...,0
7,George Gill,"Category 1: 'Risk' (Low performance, Low poten...",George gill's performance is really poor. He d...,False,True,george gills performance really poor concentra...,0
...,...,...,...,...,...,...,...
10205,Bailey Hunt,"Category 9: 'Star' (High performance, High pot...",No one performs like Bailey. I believe she wil...,False,False,one performs like bailey believe vp long ready...,8
10226,Thaddeus Burgess,"Category 9: 'Star' (High performance, High pot...",Thaddeus Burgess is a constant force within th...,True,True,thaddeus burgess constant force within team wo...,8
20022,Max Miller,"Category 9: 'Star' (High performance, High pot...",Max Miller is a a great coworker. He is dili...,True,True,max miller great coworker diligent finishing t...,8


In [13]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_text(text):
    return tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors='pt')

df['tokens'] = df['cleaned_feedback'].apply(lambda x: tokenize_text(x)).tolist()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
import torch
from torch.utils.data import Dataset, DataLoader

class EmployeeReviewDataset(Dataset):
    def __init__(self, tokens, labels):
        self.tokens = tokens
        self.labels = labels

    def __len__(self):
        return len(self.tokens)

    def __getitem__(self, idx):
        input_ids = self.tokens[idx]['input_ids'].squeeze()
        attention_mask = self.tokens[idx]['attention_mask'].squeeze()
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

In [15]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                      num_labels=9,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
from transformers import TrainerCallback

class CustomCallback(TrainerCallback):
    def __init__(self):
        self.losses = []

    def on_epoch_end(self, args, state, control, **kwargs):
      if len(state.log_history) > 0:
          print("Last log entry:", state.log_history[-1])
          if 'loss' in state.log_history[-1]:
              loss = state.log_history[-1]['loss']
              self.losses.append(loss)
              print(f"Epoch {state.epoch} - Loss: {loss:.4f}")
          else:
              print(f"Epoch {state.epoch} - Loss not available in log")
      else:
          print(f"Epoch {state.epoch} - No logs recorded")


    def save_model(self, state):
        model_dir = f"./model_epoch_{int(state.epoch)}"
        state.best_model.save_pretrained(model_dir)
        print(f"Model saved to {model_dir}")

custom_callback = CustomCallback()

In [17]:
from sklearn.model_selection import KFold
import numpy as np

kf = KFold(n_splits=5, shuffle=True, random_state=42)

X = df['tokens'].tolist()
y = df['label'].tolist()

fold = 0
all_scores = []
for train_index, val_index in kf.split(X):
    fold += 1
    print(f"Fold {fold}")

    X_train, X_val = [X[i] for i in train_index], [X[i] for i in val_index]
    y_train, y_val = [y[i] for i in train_index], [y[i] for i in val_index]

    train_dataset = EmployeeReviewDataset(X_train, y_train)
    val_dataset = EmployeeReviewDataset(X_val, y_val)

    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=16)

    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=9)

    training_args = TrainingArguments(
        output_dir=f'./results_fold_{fold}',
        evaluation_strategy="epoch",
        logging_steps=10,
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=10,
        weight_decay=0.01,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        callbacks=[custom_callback]
    )

    trainer.train()

    eval_results = trainer.evaluate()
    all_scores.append(eval_results['eval_loss'])

average_loss = np.mean(all_scores)
print(f"Ortalama Validation Loss: {average_loss}")


Fold 1


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,2.159900,2.196527
2,1.918600,1.905853
3,1.619200,1.709259
4,1.408500,1.647241
5,1.152300,1.592565
6,1.064700,1.566200
7,0.998600,1.533620
8,0.805500,1.565766
9,0.812000,1.530359
10,0.724400,1.524025


Last log entry: {'loss': 2.1599, 'grad_norm': 5.674211502075195, 'learning_rate': 1.8181818181818182e-05, 'epoch': 0.9090909090909091, 'step': 40}
Epoch 1.0 - Loss: 2.1599
Last log entry: {'loss': 1.9186, 'grad_norm': 6.559438705444336, 'learning_rate': 1.6363636363636366e-05, 'epoch': 1.8181818181818183, 'step': 80}
Epoch 2.0 - Loss: 1.9186
Last log entry: {'loss': 1.6192, 'grad_norm': 10.505173683166504, 'learning_rate': 1.4090909090909092e-05, 'epoch': 2.9545454545454546, 'step': 130}
Epoch 3.0 - Loss: 1.6192
Last log entry: {'loss': 1.4085, 'grad_norm': 10.447016716003418, 'learning_rate': 1.2272727272727274e-05, 'epoch': 3.8636363636363638, 'step': 170}
Epoch 4.0 - Loss: 1.4085
Last log entry: {'loss': 1.1523, 'grad_norm': 10.476612091064453, 'learning_rate': 1e-05, 'epoch': 5.0, 'step': 220}
Epoch 5.0 - Loss: 1.1523
Last log entry: {'loss': 1.0647, 'grad_norm': 11.970868110656738, 'learning_rate': 8.181818181818183e-06, 'epoch': 5.909090909090909, 'step': 260}
Epoch 6.0 - Loss: 1

Fold 2


...

Fold 5


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,2.135500,2.081628
2,1.944800,1.912390
3,1.623600,1.749442
4,1.471700,1.683305
5,1.314900,1.614279
6,1.129000,1.570329
7,1.017800,1.566222
8,0.809300,1.500784
9,0.778500,1.526228
10,0.707400,1.494859


Last log entry: {'loss': 2.1355, 'grad_norm': 6.026221752166748, 'learning_rate': 1.8181818181818182e-05, 'epoch': 0.9090909090909091, 'step': 40}
Epoch 1.0 - Loss: 2.1355
Last log entry: {'loss': 1.9448, 'grad_norm': 15.189579010009766, 'learning_rate': 1.6363636363636366e-05, 'epoch': 1.8181818181818183, 'step': 80}
Epoch 2.0 - Loss: 1.9448
Last log entry: {'loss': 1.6236, 'grad_norm': 9.908222198486328, 'learning_rate': 1.4090909090909092e-05, 'epoch': 2.9545454545454546, 'step': 130}
Epoch 3.0 - Loss: 1.6236
Last log entry: {'loss': 1.4717, 'grad_norm': 8.798407554626465, 'learning_rate': 1.2272727272727274e-05, 'epoch': 3.8636363636363638, 'step': 170}
Epoch 4.0 - Loss: 1.4717
Last log entry: {'loss': 1.3149, 'grad_norm': 13.694348335266113, 'learning_rate': 1e-05, 'epoch': 5.0, 'step': 220}
Epoch 5.0 - Loss: 1.3149
Last log entry: {'loss': 1.129, 'grad_norm': 18.37372398376465, 'learning_rate': 8.181818181818183e-06, 'epoch': 5.909090909090909, 'step': 260}
Epoch 6.0 - Loss: 1.12

Ortalama Validation Loss: 1.5654114007949829


In [24]:
from sklearn.metrics import classification_report

test_data = pd.read_csv(data_folder + f'/employee_review_mturk_dataset_test_v6_kaggle.csv')
test_data.set_index('id', inplace=True)
test_data.dropna(inplace=True)

test_data['cleaned_feedback'] = test_data['feedback'].apply(clean_text)

test_data['label'] = test_data['nine_box_category'].str.extract(r'Category (\d+):')[0].astype(int) - 1

test_data['tokens'] = test_data['cleaned_feedback'].apply(lambda x: tokenize_text(x)).tolist()

test_tokens = test_data['tokens'].tolist()
test_labels = test_data['label'].tolist()

test_dataset = EmployeeReviewDataset(test_tokens, test_labels)

test_dataloader = DataLoader(test_dataset, batch_size=16)
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=-1)

from sklearn.metrics import classification_report
print(classification_report(test_labels, preds, target_names=df['nine_box_category'].unique()))


                                                                         precision    recall  f1-score   support

                    Category 1: 'Risk' (Low performance, Low potential)       0.55      0.44      0.49        25
  Category 2: 'Average performer' (Moderate performance, Low potential)       0.16      0.12      0.14        25
        Category 3: 'Solid Performer' (High performance, Low potential)       0.38      0.44      0.41        25
Category 4: 'Inconsistent Player' (Low performance, Moderate potential)       0.24      0.28      0.26        25
   Category 5: 'Core Player' (Moderate performance, Moderate potential)       0.30      0.44      0.35        25
    Category 6: 'High Performer' (High performance, Moderate potential)       0.27      0.48      0.35        25
          Category 7: 'Potential Gem' (Low performance, High potential)       0.00      0.00      0.00        25
    Category 8: 'High Potential' (Moderate performance, High potential)       0.31      0.20   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
